---

# <center> Swarm Intelligence with SDK </center>

---

<center><img src="https://images.unsplash.com/photo-1516434233442-0c69c369b66d?auto=format&fit=min&crop=top&w=800&h=400"></center>

---

---
<a id="top"></a>
# Contents

This notebook is a <a href="https://www.kaggle.com/sam/halite-sdk-overview">Halite SDK</a> version of the totally cool notebook, <a href="https://www.kaggle.com/yegorbiryukov/halite-swarm-intelligence">Halite Swarm Intelligence</a>.

---

<ol>
    <li><a href="#kaggle-environments")>Install kaggle-environments</a></li>
    <li><a href="#swarm-agent")>Swarm Agent</a></li>
    <li><a href="#attack-bot")>Attack Bot</a></li>
    <li><a href="#duo-bot")>Duo Bot</a></li>
    <li><a href="#beetle-bot")>Beetle Bot</a></li>
    <li><a href="#idle-bot")>Idle Bot</a></li>
    <li><a href="#test-run")>Test Run</a></li>
</ol>

---

<a id="kaggle-environments"></a>
# Install kaggle-environments

In [ ]:
!pip install kaggle-environments --upgrade

<a href="#top">&uarr; back to top</a>

<a id="swarm-agent"></a>
# Swarm Agent
## Imports and Constants

In [ ]:
%%writefile submission.py

from kaggle_environments.envs.halite.helpers import *
from random import choice

BOARD_SIZE = None
EPISODE_STEPS = None
CONVERT_COST = None
SPAWN_COST = None

NORTH = ShipAction.NORTH
EAST = ShipAction.EAST
SOUTH = ShipAction.SOUTH
WEST = ShipAction.WEST
CONVERT = ShipAction.CONVERT
SPAWN = ShipyardAction.SPAWN

DIRECTIONS = [NORTH, EAST, SOUTH, WEST]

MOVEMENT_TACTICS = [
    [NORTH, EAST, SOUTH, WEST],
    [NORTH, WEST, SOUTH, EAST],
    [EAST, SOUTH, WEST, NORTH],
    [EAST, NORTH, WEST, SOUTH],
    [SOUTH, WEST, NORTH, EAST],
    [SOUTH, EAST, NORTH, WEST],
    [WEST, NORTH, EAST, SOUTH],
    [WEST, SOUTH, EAST, NORTH],
]
N_MOVEMENT_TACTICS = len(MOVEMENT_TACTICS)

## Swarm Controller Class
### Class Overview

```
Controller: {
    __init__(obs: Dict[str, Any], config: Dict[str, Any]) -> None
    
    clear(cell: Cell) -> bool
    hostile_ship_near(cell: Cell, halite: int) -> bool

    spawn(shipyard: Shipyard) -> None
    convert(ship: Ship) -> None
    move(ship: Ship, direction: ShipAction) -> None

    endgame(ship: Ship) -> bool
    build_shipyard(ship: Ship) -> bool
    stay_on_cell(ship: Ship) -> bool
    go_for_halite(ship: Ship) -> bool
    unload_halite(ship: Ship) -> bool
    standard_patrol(ship: Ship) -> bool
    safety_convert(ship: Ship) -> bool
    crash_shipyard(ship: Ship) -> bool

    actions_of_ships() -> None
    actions_of_shipyards() -> None
    
    next_actions() -> Dict[str, str]
}
```

### Class Implementation

In [ ]:
%%writefile -a submission.py

class Controller:
    def __init__(self, obs, config):
        """ Initialize parameters """
        global BOARD_SIZE, EPISODE_STEPS, CONVERT_COST, SPAWN_COST
        self.board = Board(obs, config)
        self.player = self.board.current_player
        self.STEP = obs.step
        
        # Define global constants
        if self.STEP == 0:
            BOARD_SIZE = config.size
            EPISODE_STEPS = config.episodeSteps
            CONVERT_COST = config.convertCost
            SPAWN_COST = config.spawnCost
            
        self.FINAL_STEP = self.STEP == EPISODE_STEPS - 2
        self.N_SHIPS = len(self.player.ships)
        self.N_SHIPYARDS = len(self.player.shipyards)

        # Cell tracking to avoid collisions of current player's ships
        self.ship_cells = set(s.cell for s in self.player.ships)
        self.ship_count = self.N_SHIPS
        self.shipyard_count = self.N_SHIPYARDS
        self.halite = self.player.halite

        # Minimum total halite before ships can convert
        self.CONVERT_THRESHOLD = CONVERT_COST + 3 * SPAWN_COST

        stocks = [c.halite for c in self.board.cells.values() if c.halite > 0]
        average_halite = int(sum(stocks) / len(stocks)) if len(stocks) > 0 else 0
        # Minimum halite a cell must have before a ship will harvest
        self.LOW_HALITE = max(average_halite // 2, 4)

        # Minimum number of ships at any time
        self.MIN_SHIPS = 10
        # Maximum number of ships to spawn
        self.MAX_SHIPS = 0
        # Increase MAX_SHIPS in first half of game only
        if self.STEP < EPISODE_STEPS // 2:
            total_ships = sum(len(p.ships) for p in self.board.players.values())
            if total_ships > 0:
                self.MAX_SHIPS = (average_halite // total_ships) * 10
        # Fix MAX_SHIPS if less than MIN_SHIPS
        self.MAX_SHIPS = max(self.MIN_SHIPS, self.MAX_SHIPS)

    def clear(self, cell):
        """ Check if cell is safe to move in """
        if (cell.ship is not None and
                cell.ship not in self.player.ships):
            return False

        if (cell.shipyard is not None and
                cell.shipyard not in self.player.shipyards):
            return False

        if cell in self.ship_cells:
            return False
        return True

    def hostile_ship_near(self, cell, halite):
        """ Check if hostile ship is one move away and has less or equal halite """
        neighbors = [cell.neighbor(d.to_point()) for d in DIRECTIONS]
        for neighbor in neighbors:
            if (neighbor.ship is not None and
                neighbor.ship not in self.player.ships and
                    neighbor.ship.halite <= halite):
                return True
        return False

    def spawn(self, shipyard):
        """ Spawn ship from shipyard """
        shipyard.next_action = SPAWN
        self.halite -= SPAWN_COST
        self.ship_count += 1
        # Cell tracking to avoid collisions of current player's ships
        self.ship_cells.add(shipyard.cell)

    def convert(self, ship):
        """ Convert ship to shipyard """
        ship.next_action = CONVERT
        self.halite -= CONVERT_COST
        self.ship_count -= 1
        self.shipyard_count += 1
        # Cell tracking to avoid collisions of current player's ships
        self.ship_cells.remove(ship.cell)

    def move(self, ship, direction):
        """ Move ship in direction """
        ship.next_action = direction
        # Cell tracking to avoid collisions of current player's ships
        if direction is not None:
            d_cell = ship.cell.neighbor(direction.to_point())
            self.ship_cells.remove(ship.cell)
            self.ship_cells.add(d_cell)
            
    def endgame(self, ship):
        """" Final step: convert if possible """
        if (self.FINAL_STEP and
                ship.halite >= CONVERT_COST):
            self.convert(ship)
            return True
        return False
    
    def build_shipyard(self, ship):
        """ Convert to shipyard if necessary """
        if (self.shipyard_count == 0 and
              self.ship_count < self.MAX_SHIPS and
              self.STEP < EPISODE_STEPS // 2 and
              self.halite + ship.halite >= self.CONVERT_THRESHOLD and
              not self.hostile_ship_near(ship.cell, ship.halite)):
            self.convert(ship)
            return True
        return False
    
    def stay_on_cell(self, ship):
        """ Stay on current cell if profitable and safe """
        if (ship.cell.halite > self.LOW_HALITE and
              not self.hostile_ship_near(ship.cell, ship.halite)):
            ship.next_action = None
            return True
        return False
    
    def go_for_halite(self, ship):
        """ Ship will move to safe cell with largest amount of halite """
        neighbors = [(d, ship.cell.neighbor(d.to_point())) for d in DIRECTIONS]
        candidates = [(d, c) for d, c in neighbors if self.clear(c) and
                      not self.hostile_ship_near(c, ship.halite) and
                      c.halite > self.LOW_HALITE]

        if candidates:
            stocks = [c.halite for d, c in candidates]
            max_idx = stocks.index(max(stocks))
            direction = candidates[max_idx][0]
            self.move(ship, direction)
            return True
        return False

    def unload_halite(self, ship):
        """ Unload ship's halite if it has any and vacant shipyard is near """
        if ship.halite > 0:
            for d in DIRECTIONS:
                d_cell = ship.cell.neighbor(d.to_point())

                if (d_cell.shipyard is not None and
                        self.clear(d_cell)):
                    self.move(ship, d)
                    return True
        return False

    def standard_patrol(self, ship):
        """ Ship will move in circles clockwise or counterclockwise if safe"""
        # Choose movement tactic
        i = int(ship.id.split("-")[0]) % N_MOVEMENT_TACTICS
        directions = MOVEMENT_TACTICS[i]
        # Select initial direction
        n_directions = len(directions)
        j = (self.STEP // BOARD_SIZE) % n_directions
        # Move to first safe direction found
        for _ in range(n_directions):
            direction = directions[j]
            d_cell = ship.cell.neighbor(direction.to_point())
            # Check if direction is safe
            if (self.clear(d_cell) and
                    not self.hostile_ship_near(d_cell, ship.halite)):
                self.move(ship, direction)
                return True
            # Try next direction
            j = (j + 1) % n_directions
        # No safe direction
        return False

    def safety_convert(self, ship):
        """ Convert ship if not on shipyard and hostile ship is near """
        if (ship.cell.shipyard is None and
            self.hostile_ship_near(ship.cell, ship.halite) and
                ship.halite >= CONVERT_COST):
            self.convert(ship)
            return True
        return False

    def crash_shipyard(self, ship):
        """ Crash into opponent shipyard """
        for d in DIRECTIONS:
            d_cell = ship.cell.neighbor(d.to_point())

            if (d_cell.shipyard is not None and
                    d_cell.shipyard not in self.player.shipyards):
                self.move(ship, d)
                return True
        return False

    def actions_of_ships(self):
        """ Next actions of every ship """
        for ship in self.player.ships:
            # Act according to first acceptable tactic
            if self.endgame(ship):
                continue
            if self.build_shipyard(ship):
                continue
            if self.stay_on_cell(ship):
                continue
            if self.go_for_halite(ship):
                continue
            if self.unload_halite(ship):
                continue
            if self.standard_patrol(ship):
                continue
            if self.safety_convert(ship):
                continue
            if self.crash_shipyard(ship):
                continue
            # Default random action
            self.move(ship, choice(DIRECTIONS + [None]))

    def actions_of_shipyards(self):
        """ Next actions of every shipyard """
        # Spawn ships from every shipyard if possible
        for shipyard in self.player.shipyards:
            if (self.ship_count < self.MAX_SHIPS and
                self.halite >= SPAWN_COST and
                not self.FINAL_STEP and
                    self.clear(shipyard.cell)):
                self.spawn(shipyard)
            else:
                shipyard.next_action = None

    def next_actions(self):
        """ Perform next actions for current player """
        self.actions_of_ships()
        self.actions_of_shipyards()
        return self.player.next_actions


## Main Agent Function

In [ ]:
%%writefile -a submission.py

def agent(obs, config):
    controller = Controller(obs, config)
    return controller.next_actions()

<a href="#top">&uarr; back to top</a>

<a id="attack-bot"></a>
# Attack Bot

This bot tries to attack the swarm.

In [ ]:
%%writefile attack_bot.py
from kaggle_environments.envs.halite.helpers import *

DIRECTIONS = [ShipAction.NORTH, ShipAction.EAST, 
              ShipAction.SOUTH, ShipAction.WEST]

def agent(obs, config):
    board = Board(obs, config)
    player = board.current_player
    next_cells = set()

    def safe(c, halite):
        if c in next_cells:
            return False
        good = player.ships + player.shipyards + [None]
        if c.shipyard not in good:
            return False
        for n in [c, c.north, c.east, c.south, c.west]:
            if (n.ship not in good) and (n.ship.halite <= halite):
                return False
        return True

    def next_action(s, action):
        s.next_action = action
        if action in DIRECTIONS:
            next_cells.add(s.cell.neighbor(action.to_point()))
        elif action is None:
            next_cells.add(s.cell)

    yields = [int(c.halite) for c in board.cells.values() if c.halite > 0]
    min_halite = max(4, sum(yields) // max(1, len(yields)) // 2)
    max_shipyards = 10
    
    for ship in player.ships:
        cell = ship.cell
        ship.next_action = None
        
        if len(player.shipyards) == 0 and safe(cell, ship.halite):
            next_action(ship, ShipAction.CONVERT)
            continue
            
        if (obs.step == config.episodeSteps - 2 and 
            ship.halite >= config.convertCost):
            next_action(ship, ShipAction.CONVERT)
            continue
            
        if (obs.step > config.episodeSteps - 20 and 
            len(player.shipyards) > 0 and ship.halite > 0):
            i = sum(int(k) for k in ship.id.split("-")) % len(player.shipyards)
            dx, dy = player.shipyards[i].position - ship.position
            if dx > 0 and safe(cell.east, ship.halite):
                next_action(ship, ShipAction.EAST)
            elif dx < 0 and safe(cell.west, ship.halite):
                next_action(ship, ShipAction.WEST)
            elif dy > 0 and safe(cell.north, ship.halite):
                next_action(ship, ShipAction.NORTH)
            elif dy < 0 and safe(cell.south, ship.halite):
                next_action(ship, ShipAction.SOUTH)
            if ship.next_action in DIRECTIONS:
                continue
        
        for d in DIRECTIONS:
            neighbor = cell.neighbor(d.to_point())
            if (neighbor.ship is not None and 
                neighbor.ship not in player.ships and
                safe(neighbor, ship.halite)):
                next_action(ship, d)
                break
        if ship.next_action in DIRECTIONS:
            continue
                
        if cell.halite > min_halite and safe(cell, ship.halite):
            next_action(ship, None)
            continue
            
        if (ship.halite > config.convertCost * 4 and 
            len(player.shipyards) < max_shipyards and safe(cell, ship.halite)):
            next_action(ship, ShipAction.CONVERT)
            continue
                
        neighbors = [cell.neighbor(d.to_point()) for d in DIRECTIONS]
        max_halite = max([0] + [n.halite for n in neighbors if safe(n, ship.halite)])
        i = sum(int(k) for k in ship.id.split("-")) * config.size
        j = ((i + obs.step) // config.size) % 4
        safe_list = []
        for _ in range(4):
            d = DIRECTIONS[j]
            n = cell.neighbor(d.to_point())
            if safe(n, ship.halite):
                if ((n.halite > min_halite and n.halite == max_halite) or 
                    (ship.halite > 20 and n.shipyard in player.shipyards)):
                    next_action(ship, d)
                    break
                safe_list.append(d)
            j = (j + 1) % 4
        else:
            if safe_list:
                next_action(ship, safe_list[0])
            elif safe(cell, ship.halite):
                next_action(ship, None)
            elif ship.halite >= config.convertCost:
                next_action(ship, ShipAction.CONVERT)
            else:
                next_action(ship, None)
                
    max_ships = min(50, len(player.ships) +
                    player.halite // config.spawnCost // 
                    max(1, len(player.shipyards)))
    
    for shipyard in player.shipyards:
        if len(player.ships) == 0:
            shipyard.next_action = ShipyardAction.SPAWN
            
        elif (len(player.ships) < max_ships and
              obs.step < config.episodeSteps - 50 and
              safe(shipyard.cell, ship.halite)):
            shipyard.next_action = ShipyardAction.SPAWN
            
    return player.next_actions

<a href="#top">&uarr; back to top</a>

<a id="duo-bot"></a>
# Duo Bot

This bot maintains a fleet of two ships.


In [ ]:
%%writefile duo_bot.py
from kaggle_environments.envs.halite.helpers import *

DIRECTIONS = [ShipAction.NORTH, ShipAction.EAST, 
              ShipAction.SOUTH, ShipAction.WEST]

def agent(obs, config):
    board = Board(obs, config)
    player = board.current_player
    next_cells = set()

    def safe(c, halite):
        if c in next_cells:
            return False
        good = player.ships + player.shipyards + [None]
        if c.shipyard not in good:
            return False
        for n in [c, c.north, c.east, c.south, c.west]:
            if (n.ship not in good) and (n.ship.halite <= halite):
                return False
        return True

    def next_action(s, action):
        s.next_action = action
        if action in DIRECTIONS:
            next_cells.add(s.cell.neighbor(action.to_point()))
        elif action is None:
            next_cells.add(s.cell)

    yields = [int(c.halite) for c in board.cells.values() if c.halite > 0]
    avg_halite = sum(yields) // max(1, len(yields))
    min_halite = max(4, avg_halite // 2)
    max_halite = 4 * min_halite
    
    for ship in player.ships:
        cell = ship.cell
        ship.next_action = None
        
        if len(player.shipyards) == 0 and safe(cell, ship.halite):
            next_action(ship, ShipAction.CONVERT)
            continue
            
        if (obs.step == config.episodeSteps - 2 and 
            ship.halite >= config.convertCost):
            next_action(ship, ShipAction.CONVERT)
            continue
            
        if (ship.id == player.ships[0].id and 
            ship.halite > max_halite and 
            len(player.shipyards) > 0):
            i = sum(int(k) for k in ship.id.split("-")) % len(player.shipyards)
            dx, dy = player.shipyards[i].position - ship.position
            if dx > 0 and safe(cell.east, ship.halite):
                next_action(ship, ShipAction.EAST)
            elif dx < 0 and safe(cell.west, ship.halite):
                next_action(ship, ShipAction.WEST)
            elif dy > 0 and safe(cell.north, ship.halite):
                next_action(ship, ShipAction.NORTH)
            elif dy < 0 and safe(cell.south, ship.halite):
                next_action(ship, ShipAction.SOUTH)
            if ship.next_action in DIRECTIONS:
                continue
                
        if cell.halite > min_halite and safe(cell, ship.halite):
            next_action(ship, None)
            continue
                            
        neighbors = [cell.neighbor(d.to_point()) for d in DIRECTIONS]
        max_halite = max([0] + [n.halite for n in neighbors if safe(n, ship.halite)])
        i = sum(int(k) for k in ship.id.split("-")) * config.size
        j = ((i + obs.step) // config.size) % 4
        safe_list = []
        for _ in range(4):
            d = DIRECTIONS[j]
            n = cell.neighbor(d.to_point())
            if safe(n, ship.halite):
                if n.halite > min_halite and n.halite == max_halite:
                    next_action(ship, d)
                    break
                safe_list.append(d)
            j = (j + 1) % 4
        else:
            if safe_list:
                next_action(ship, safe_list[0])
            elif safe(cell, ship.halite):
                next_action(ship, None)
            elif ship.halite >= config.convertCost:
                next_action(ship, ShipAction.CONVERT)
            else:
                next_action(ship, None)
    
    for shipyard in player.shipyards:
        if len(player.ships) < 2:
            shipyard.next_action = ShipyardAction.SPAWN
            
    return player.next_actions

<a href="#top">&uarr; back to top</a>

<a id="beetle-bot"></a>
# Beetle Bot

This is a bot with one ship and one shipyard which is described in the notebook: <a href="https://www.kaggle.com/benzyx/make-sure-you-can-beat-this-baseline-idle-bot">Make sure you can beat this: Baseline Idle Bot</a>.


In [ ]:
%%writefile beetle_bot.py
from kaggle_environments.envs.halite.helpers import *

def agent(obs, config):    
    board = Board(obs, config)
    player = board.current_player
    
    for ship in player.ships:
        if len(player.shipyards) == 0:
            ship.next_action = ShipAction.CONVERT
        
    for shipyard in player.shipyards:
        if len(player.ships) == 0:
            shipyard.next_action = ShipyardAction.SPAWN
            
    return player.next_actions

<a href="#top">&uarr; back to top</a>

<a id="idle-bot"></a>
# Idle Bot

This bot does nothing.


In [ ]:
%%writefile idle_bot.py
def agent(obs, config):            
    return {}

<a href="#top">&uarr; back to top</a>

<a id="test-run"></a>
# Test Run

In [ ]:
import random
from kaggle_environments import make

def run_test():
    agent_zoo = ["random", "idle_bot.py", "beetle_bot.py", "duo_bot.py", "attack_bot.py"]
    agents = random.sample(agent_zoo, 3)
    agents.insert(random.randint(0,3), "submission.py")
    print("Agents:", agents)
    
    environment = make("halite", configuration={"episodeSteps": 200}, debug=True)
    environment.run(agents)
    environment.render(mode="ipython", width=640, height=480)

In [ ]:
%%time
run_test()

In [ ]:
%%time
run_test()

In [ ]:
%%time
run_test()

In [ ]:
%%time
run_test()

In [ ]:
%%time
run_test()

<a href="#top">&uarr; back to top</a>